# AWSome Builder III (Mehdi Salehi)
Containerize a COBOL app and move it to AWS

### The existing legacy enviroment:

In [192]:
%%bash

export AIX_HOST="52.117.41.211"

export AIX_USER="ab3user"

export CMD="uname; oslevel -s; prtconf 2>/dev/null | head -5"

ssh ab3user@$AIX_HOST $CMD


AIX
7200-04-01-1939
System Model: IBM,9009-22A
Machine Serial Number: 78BABE0
Processor Type: PowerPC_POWER9
Processor Implementation Mode: POWER 9
Processor Version: PV_9_Compat


### Checking the COBOL Compiler on AIX host:

In [63]:
%%bash

export AIX_HOST="52.117.41.211"

export AIX_USER="ab3user"

export CMD="lslpp -l cobol.rte"

ssh $AIX_USER@$AIX_HOST $CMD

  Fileset                      Level  State      Description         
  ----------------------------------------------------------------------------
Path: /usr/lib/objrepos
  cobol.rte                  5.1.0.0  COMMITTED  IBM COBOL for AIX Runtime


### Running a sample COBOL application on AIX (Edit the code in modern developement tools):
https://github.com/awsengineer/awsome-builder/blob/main/AWSome.cbl

In [193]:
%%bash
export AIX_HOST="52.117.41.211"
export AIX_USER="ab3user"
export APP_DIR="/home/cobol"
export COBOL_FILE="AWSome.cbl"

#cat <<EOF >$COBOL_FILE
#       IDENTIFICATION DIVISION.
#      PROGRAM-ID.  ABIII.
#
#       PROCEDURE DIVISION.
#       DisplayPrompt.
#           DISPLAY "Woohoo :) I am a COBOL code running on AIX!".
#           STOP RUN.
#EOF

git pull > /dev/null 2>&1

scp $COBOL_FILE $AIX_USER@$AIX_HOST:$APP_DIR && echo -e "\nThe COBOL code has been uploaded to the AIX host successfully."

export CMD="echo -e '\nA Quick look at the code on AIX host:\n' ;cat ${APP_DIR}/${COBOL_FILE}"

ssh $AIX_USER@$AIX_HOST $CMD


The COBOL code has been uploaded to the AIX host successfully.

A Quick look at the code on AIX host:

       IDENTIFICATION DIVISION.
       PROGRAM-ID.  ABIII.
       PROCEDURE DIVISION.
       DisplayPrompt.
           DISPLAY "Hiii! I am a legacy COBOL code running on AIX.".
           STOP RUN.

### Compile the COBOL code on AIX:

In [195]:
%%bash
export AIX_HOST="52.117.41.211"
export AIX_USER="ab3user"
export APP_DIR="/home/cobol"
export COBOL_CODE="$APP_DIR/AWSome.cbl"
export EXEC_FILE="$APP_DIR/AWSome.exe"

ssh $AIX_USER@${AIX_HOST} "

echo -e 'Remove the exising complied app...\n'
rm -f $EXEC_FILE  2>/dev/null 

echo -e 'Compile the COBOL code...\n'

cob2 $COBOL_CODE -o $EXEC_FILE #>/dev/null

if [ -f $EXEC_FILE ]
then 
    echo -e '\nCheck the output binary file:\n'
    file $EXEC_FILE
    echo -e '\nRun the executable file:\n'
    $EXEC_FILE;  
else
    echo "Compile error" 
fi"


Remove the exising complied app...

Compile the COBOL code...

PP 5724-Z87 IBM COBOL for AIX  5.1.0 in progress ...
End of compilation 1,  program ABIII,  no statements flagged.

Check the output binary file:

/home/cobol/AWSome.exe: executable (RISC System/6000) or object module not stripped

Run the executable file:

Hiii! I am a legacy COBOL code running on AIX.


### Run the same code on a Linux container:

In [90]:
%%bash

if [ -f AWSome.cbl ]
then 
    echo -e "\nThe contents of the Dockerfile:"
    echo -e "-------------------------------\n"
    cat Dockerfile
else
    echo "The COBOL source code (AWSome.cbl) does not exist."
fi


The contents of the Dockerfile:
-------------------------------

FROM debian:latest

RUN apt-get update && apt-get install -y open-cobol

RUN mkdir /src

COPY ./AWSome.cbl /src

RUN cd /src && cobc -free -x -o AWSome.exe AWSome.cbl

CMD "./src/AWSome.exe"


### Create the Docker image:

In [201]:
%%bash

docker -v >/dev/null 2>&1

if [ $? -eq "0" ] && [ -f ./Dockerfile ]
then
    docker build -t ezzobad/awsome-builder:ABIII .
else
    echo -e "\nDocker environemnt is not ready."
fi






Step 1/6 : FROM debian:latest
 ---> e7d08cddf791
Step 2/6 : RUN apt-get update && apt-get install -y open-cobol
 ---> Using cache
 ---> e4f22fb2276d
Step 3/6 : RUN mkdir /src
 ---> Using cache
 ---> 7eaa0ae317ac
Step 4/6 : COPY ./AWSome.cbl /src
 ---> Using cache
 ---> 5b375bbe7575
Step 5/6 : RUN cd /src && cobc -free -x -o AWSome.exe AWSome.cbl
 ---> Using cache
 ---> c019733701c1
Step 6/6 : CMD "./src/AWSome.exe"
 ---> Using cache
 ---> c10e2e3eeecb
Successfully built c10e2e3eeecb
Successfully tagged ezzobad/awsome-builder:ABIII


### Test the Docker image locally:

In [204]:
%%bash

echo -e "\nThe details of the local Docker image:\n"
docker image ls ezzobad/awsome-builder:ABIII

echo -e "\nNow run a container based on the recently-created image:\n"
docker run ezzobad/awsome-builder:ABIII



The details of the local Docker image:

REPOSITORY               TAG                 IMAGE ID            CREATED             SIZE
ezzobad/awsome-builder   ABIII               c10e2e3eeecb        2 minutes ago       273MB

Now run a container based on the recently-created image:

Hiii! I am a legacy COBOL code running on AIX.


### Push the image to a Docker repository:
https://hub.docker.com/repository/docker/ezzobad/awsome-builder

In [205]:
%%bash

docker push ezzobad/awsome-builder:ABIII


The push refers to repository [docker.io/ezzobad/awsome-builder]
52d42724ee23: Preparing
dace43b25d6c: Preparing
7d540c05a6b1: Preparing
3cf02b66d37a: Preparing
4762552ad7d8: Preparing
4762552ad7d8: Layer already exists
dace43b25d6c: Pushed
7d540c05a6b1: Pushed
52d42724ee23: Pushed
3cf02b66d37a: Pushed
ABIII: digest: sha256:6e68b20a9371400d544462a80dc5d602c997a287e9d8a72436c8665c02597700 size: 1363


### Verify the Docker repository:

In [208]:
%%bash

echo -e "Delete the local image:\n"
docker image rm ezzobad/awsome-builder:ABIII --force

echo -e "\nNow run a container. This will pull the image from the repo:\n"
docker run ezzobad/awsome-builder:ABIII 2>/dev/null

Delete the local image:

Untagged: ezzobad/awsome-builder:ABIII
Untagged: ezzobad/awsome-builder@sha256:6e68b20a9371400d544462a80dc5d602c997a287e9d8a72436c8665c02597700
Deleted: sha256:c10e2e3eeecb21e13c7b08ec0de29872d02db57e763a6720bf7de532af6fb68d

Now run a container. This will pull the image from the repo:

Hiii! I am a legacy COBOL code running on AIX.


### Openshift environment on AWS:

https://cloud.redhat.com/openshift/install/aws/installer-provisioned

AWS OpenShift Installer on Mac:
https://mirror.openshift.com/pub/openshift-v4/clients/ocp/latest/openshift-install-mac.tar.gz 
    
OpenShift Client (OC) for Mac:
https://mirror.openshift.com/pub/openshift-v4/clients/ocp/latest/openshift-client-mac.tar.gz 



### Explore the OpenShift cluster from CLI:

In [209]:
%%bash

export PATH=/home/ec2-user/anaconda3/bin:$PATH:/home/ec2-user/bin
        
oc login --server https://api.awsome-openshift-cluster.awsome-builder.com:6443 \
--username kubeadmin --password QUPhE-eRKET-GojYB-aHJKu > /dev/null 2>&1

oc new-project ab3project > /dev/null 2>&1

echo -e "\nCheck the cluster nodes:\n"
oc get nodes

echo -e "\nCheck the cluster resources:\n"
kubectl top nodes

echo -e "\nTweek the number of worker nodes (the hard way):\n"
echo 'oc edit machinesets.machine.openshift.io awsome-openshift-clus-f94d7-worker-us-west-1b -n openshift-machine-api'

echo -e "\nCheck the EC2 instances:\n"
echo 'https://tiny.amazon.com/48qywull/usweconsawsamazec2v2home'

echo -e "\nCheck the number of machines:\n"
oc get machinesets.machine.openshift.io -n openshift-machine-api

# kubectl run -it ab3 --image=ezzobad/awsome-builder:ABIII -- bash    
# kubectl exec ab3 -- ./src/AWSome.exe



Check the cluster nodes:

NAME                                         STATUS   ROLES    AGE    VERSION
ip-10-0-164-121.us-west-1.compute.internal   Ready    master   23d    v1.19.0+43983cd
ip-10-0-176-128.us-west-1.compute.internal   Ready    master   23d    v1.19.0+43983cd
ip-10-0-181-167.us-west-1.compute.internal   Ready    worker   164m   v1.19.0+43983cd
ip-10-0-186-141.us-west-1.compute.internal   Ready    worker   156m   v1.19.0+43983cd
ip-10-0-190-130.us-west-1.compute.internal   Ready    worker   23d    v1.19.0+43983cd
ip-10-0-222-169.us-west-1.compute.internal   Ready    master   23d    v1.19.0+43983cd
ip-10-0-235-231.us-west-1.compute.internal   Ready    worker   23d    v1.19.0+43983cd

Check the cluster resources:

NAME                                         CPU(cores)   CPU%   MEMORY(bytes)   MEMORY%   
ip-10-0-164-121.us-west-1.compute.internal   599m         17%    3912Mi          27%       
ip-10-0-176-128.us-west-1.compute.internal   1030m        29%    6139Mi       

### Run a test container from the GUI Console
https://tiny.amazon.com/hjtrscbz/oautappsawsoawsologi 

### Run the COBOL app from OpenShift CLI:

In [210]:
%%bash
export PATH=/home/ec2-user/anaconda3/bin:$PATH:/home/ec2-user/bin

oc delete pod testpod --force 2>/dev/null
oc run testpod --image=ezzobad/awsome-builder:ABIII --restart Never
sleep 3
echo 
oc logs testpod 

pod/testpod created

Hiii! I am a legacy COBOL code running on AIX.


# Nest step: Automation of Docker image creation

In [187]:
%%bash
export PATH=/home/ec2-user/anaconda3/bin:$PATH:/home/ec2-user/bin
export COBOL_FILE="AWSome.cbl"

git pull 

cat <<EOF >$COBOL_FILE
      IDENTIFICATION DIVISION.
      PROGRAM-ID.  ABIII.

       PROCEDURE DIVISION.
       DisplayPrompt.
           DISPLAY "OK! Let's automate!".
           STOP RUN.
EOF

#git add $COBOL_FILE

git commit -a -m "Let's trigger Docker image build"
git push

# Create a Dockerhub build to trigger an action when Dockerfile is updated on Github
# Use the ssh-key of the Dockerhub repository to authenticate with Github
echo -e "https://hub.docker.com/repository/docker/ezzobad/awsome-builder/builds/edit\n"


Already up to date.
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
	.ipynb_checkpoints/AWSome-checkpoint.cbl

nothing added to commit but untracked files present
https://hub.docker.com/repository/docker/ezzobad/awsome-builder/builds/edit



Everything up-to-date


# Create a Pod based on the lastest image:

In [190]:
%%bash
export PATH=/home/ec2-user/anaconda3/bin:$PATH:/home/ec2-user/bin


echo -e "Delete the old Pod if exists...\n"
oc delete pod newpod --force 2>/dev/null

echo -e "Create a Pod based on the latest image...\n"
oc run newpod --image=ezzobad/awsome-builder:ABIII \
--image-pull-policy=Always --restart=Never 

echo -e "Wait until the Pod gets ready...\n"
while [ `oc get pod newpod -o jsonpath="{.status.phase}"` != "Succeeded" ]
do 
sleep 1; 
done

echo -e "Get the application's log:\n"
oc logs newpod

Delete the old Pod if exists...

pod "newpod" force deleted
Create a Pod based on the latest image...

pod/newpod created
Wait until the Pod gets ready...

Get the application's log:

OK! Let's automate!
